In [4]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip

--2020-05-24 07:55:20--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4b8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

100%[====================================>] 5,828,358,084 13.4MB/s   in 7m 9s  

2020-05-24 08:02:30 (12.9 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  


In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath(''))

In [2]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

from facebook_hateful_memes_detector import read_json_lines_into_df
from facebook_hateful_memes_detector import FasttextPooledModel
from facebook_hateful_memes_detector.preprocessing import TextImageDataset




In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dev = read_json_lines_into_df('../data/dev.jsonl')
train = read_json_lines_into_df('../data/train.jsonl')
test = read_json_lines_into_df('../data/test.jsonl')

train_text = list(train.text)
dev_text = list(dev.text)
test_text = list(test.text)

train_img = list(map(lambda l: "../data/"+l,train.img))
test_img = list(map(lambda l: "../data/"+l,test.img))
dev_img = list(map(lambda l: "../data/"+l,dev.img))

# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(categories=[[0, 1]],handle_unknown='error', sparse=False)
train_labels = torch.tensor(train.label)
dev_labels = torch.tensor(dev.label)

train_text = np.array(train_text)
dev_text = np.array(dev_text)
test_text = np.array(test_text)

train_img = np.array(train_img)
test_img = np.array(test_img)
dev_img = np.array(dev_img)


In [5]:
from PIL import Image
channel3 = [np.array(Image.open(im)).shape[2] == 3 for im in train_img]
all(channel3)



IndexError: tuple index out of range

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def my_collate(batch):
    text = [item[0] for item in batch]
    image = torch.stack([item[1] for item in batch])
    label = [item[2] for item in batch]
    label = torch.LongTensor(label) if label[0] is not None else label
    return [text, image, label]

In [ ]:
from tqdm import tqdm_notebook as tqdm, trange
training_fold_dataset = TextImageDataset(train_text, train_img, train_labels, image_transform=preprocess)
testing_fold_dataset = TextImageDataset(test_text, test_img, image_transform=preprocess)
train_loader = DataLoader(training_fold_dataset, batch_size=64, collate_fn=my_collate, 
                          shuffle=False,num_workers=8, pin_memory=True)
# val_loader = DataLoader(testing_fold_dataset, batch_size=128, collate_fn=my_collate, 
#                         shuffle=False,num_workers=8, pin_memory=True)


print("Start Iterating")
epochs = 10
for epoch in trange(epochs):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    print("Epoch = ", epoch)

    # Training the model
    counter = 0
    for i, (texts, images, labels) in tqdm(enumerate(train_loader)):
        print("Epoch = ", epoch, "Sub = ", i)
#     for texts, images, labels in val_loader:
#         pass
        
    

In [ ]:
try:
    with tqdm(...) as t:
        for i in t:
            ...
except KeyboardInterrupt|Exception:
    t.close()
    raise
t.close()

In [9]:
from sklearn.model_selection import KFold
from tqdm import tqdm_notebook as tqdm, trange
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(train_labels):
    training_text_fold = train_text[train_index]
    testing_text_fold = train_text[test_index]
    training_img_fold = train_img[train_index]
    testing_img_fold = train_img[test_index]
    training_labels_fold = train_labels[train_index]
    testing_labels_fold = train_labels[test_index]
    training_fold_dataset = TextImageDataset(training_text_fold, training_img_fold, training_labels_fold, image_transform=preprocess)
    testing_fold_dataset = TextImageDataset(testing_text_fold, testing_img_fold, image_transform=preprocess)
    
    train_loader = DataLoader(training_fold_dataset, batch_size=512, collate_fn=my_collate, 
                              shuffle=True,num_workers=32, pin_memory=True)
    val_loader = DataLoader(testing_fold_dataset, batch_size=512, collate_fn=my_collate, 
                            shuffle=True,num_workers=32, pin_memory=True)
    
    model = FasttextPooledModel(300, 1024, 2, "crawl-300d-2M-subword.bin",)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    epochs = 5
    try:
        with tqdm_notebook(list(range(epochs))) as epochs:
            for epoch in epochs:
                train_loss = 0
                val_loss = 0
                accuracy = 0

                # Training the model
                _ = model.train()
                counter = 0
                with tqdm(train_loader) as data_batch:
                    for texts, images, labels in data_batch:
                        optimizer.zero_grad()
                        logits, loss = model.forward(texts, images, labels)
                        loss.backward()
                        optimizer.step()
                        train_loss += loss.item()*labels.size(0)


                # Evaluating the model
                _ = model.eval()
                with torch.no_grad():
                    for texts, images, labels in val_loader:
                        logits, valloss = model.forward(texts, images, labels)
                        val_loss += valloss.item()*labels.size(0)


                        # output = torch.softmax(logits)
                        top_p, top_class = logits.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()


                # Get the average loss for the entire epoch
                train_loss = train_loss/len(train_loader.dataset)
                valid_loss = val_loss/len(val_loader.dataset)
                print('Accuracy: ', accuracy/len(val_loader))
                print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    except (KeyboardInterrupt, Exception) as e:
        epochs.close()
        data_batch.close()
        raise
    epochs.close()
    data_batch.close()

FasttextPooledModel(
  (classifier): Sequential(
    (0): Linear(in_features=300, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=1024, out_features=2, bias=True)
  )
  (loss): CrossEntropyLoss()
)

  0%|          | 0/5 [00:00<?, ?it/s]/home/ahemf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/models/text_models/FasttextPooled.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


 20%|██        | 1/5 [00:37<02:28, 37.02s/it]

Accuracy:  0.9891387224197388
Epoch: 0 	Training Loss: 0.662548 	Validation Loss: 0.513086


...

Accuracy:  0.9193978607654572
Epoch: 2 	Training Loss: 0.616479 	Validation Loss: 0.452085


 80%|████████  | 4/5 [03:00<00:45, 45.06s/it]

Accuracy:  0.8941263407468796
Epoch: 3 	Training Loss: 0.607014 	Validation Loss: 0.446993


100%|██████████| 5/5 [03:45<00:00, 45.05s/it]

Accuracy:  0.8472632467746735
Epoch: 4 	Training Loss: 0.596708 	Validation Loss: 0.480677
